In [1]:
# imports
import pandas as pd
import pickle as pkl
from sklearn.preprocessing import OneHotEncoder, KBinsDiscretizer, MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
#
from moduleDatabase import DatabaseMethods

In [2]:
# class instantiation
db = DatabaseMethods()

In [3]:
df = db.fetch("SELECT * FROM v8")
df = db.separate_target_class(df, "moca_pre_binary_binned")

In [4]:
features = ['education', 'avg_gr_time_in_gs', 'family_med_history', 'total_gr_in_gs']
X = df[features]
y = df['target_class']

In [5]:
scaler = StandardScaler()
discretizer = KBinsDiscretizer(n_bins=6, encode='ordinal', strategy='quantile')

# include make_column_transformer in case we use one hot encoding
# ct = make_column_transformer(
#     (discretizer, ['age','avg_gr_time_win_gr_in_gs', 'avg_gr_time_win_gr_in_gs', 'avg_gr_time_in_gs', 
#                    'total_win_gr_points_in_gs', 'total_gr_in_gs', 'total_success_rounds_in_session']),
#     (scaler, ['age', 'education','laptop_usage', 'smartphone_usage', 'family_med_history', 'exercising', 
#               'hypertension', 'total_gr_in_gs', 'total_success_rounds_in_session', 
#               'total_win_gr_points_in_gs', 'avg_gr_time_in_gs', 'avg_gr_time_win_gr_in_gs']),
#     remainder='passthrough')


# TODO INCLUDE PCA

clf = KNeighborsClassifier(n_neighbors=15)

In [6]:
pipe = make_pipeline(scaler, discretizer, clf)

In [7]:
# train model
model = pipe.fit(X, y)

# save model for service
pathFileName = 'models/model.joblib'
pkl.dump(model, open(pathFileName, 'wb'))
print(type(model))

<class 'sklearn.pipeline.Pipeline'>


C:\Users\christos\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
C:\Users\christos\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 2 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


In [8]:
# test that the created model can actually make a prediction
test = pd.DataFrame({'education':[2], 'avg_gr_time_in_gs':[15], 'family_med_history':[0], 'total_gr_in_gs':[10]})
X_new = test[features]
print(pipe.predict(X_new))

[1]
